In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.arima.model import ARIMA

# SQL 실행 및 데이터프레임 로드
df = pd.read_csv(path)

# 1. 데이터 준비
# 데이터 로드 및 정제는 이전 코드 사용
# df_recent: 최근 5년치 데이터를 포함하는 DataFrame
# df: 전체 데이터

# 데이터 정리
df['net_contracts'] = df['contract_issued'] - df['contract_expired']

# 2. 시각화 1: 월별 계약 발생 및 종료 건수 추세
plt.figure(figsize=(12, 6))
plt.plot(df_recent.index, df_recent['contract_issued'], label='Contracts Issued', marker='o')
plt.plot(df_recent.index, df_recent['contract_expired'], label='Contracts Expired', marker='o', linestyle='--')
plt.title('Monthly Contracts Issued and Expired (Last 5 Years)')
plt.xlabel('Month')
plt.ylabel('Number of Contracts')
plt.legend()
plt.grid()
plt.show()

# 3. 시각화 2: 월별 순 계약 건수(Net Contracts) 추세
plt.figure(figsize=(12, 6))
plt.bar(df_recent.index, df_recent['net_contracts'], color='skyblue')
plt.axhline(0, color='red', linestyle='--', linewidth=1)
plt.title('Monthly Net Contracts (Issued - Expired)')
plt.xlabel('Month')
plt.ylabel('Net Contracts')
plt.grid(axis='y')
plt.show()

# 4. 시각화 3: Boxplot으로 계약 발생 및 종료 분포 확인
plt.figure(figsize=(12, 6))
sns.boxplot(data=df_recent[['contract_issued', 'contract_expired']])
plt.title('Distribution of Contracts Issued and Expired (Last 5 Years)')
plt.ylabel('Number of Contracts')
plt.xticks([0, 1], ['Contracts Issued', 'Contracts Expired'])
plt.show()

# 5. 시각화 4: Heatmap으로 월별 발생/종료 건수 상관관계 분석
correlation = df_recent[['contract_issued', 'contract_expired', 'net_contracts']].corr()
plt.figure(figsize=(8, 6))
sns.heatmap(correlation, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Correlation Between Contracts Metrics')
plt.show()

# 6. 시각화 5: 월별 계약 발생 및 종료 건수 비교 (Stacked Bar Chart)
plt.figure(figsize=(12, 6))
plt.bar(df_recent.index, df_recent['contract_issued'], label='Contracts Issued', color='blue')
plt.bar(df_recent.index, df_recent['contract_expired'], label='Contracts Expired', color='orange', bottom=df_recent['contract_issued'])
plt.title('Monthly Contracts Issued and Expired (Stacked)')
plt.xlabel('Month')
plt.ylabel('Number of Contracts')
plt.legend()
plt.grid(axis='y')
plt.show()

# 7. 시각화 6: 월별 데이터의 시즌성 분석 (월별 평균 발생 및 종료 건수)
df_recent['month_only'] = df_recent.index.month
monthly_avg = df_recent.groupby('month_only')[['contract_issued', 'contract_expired']].mean()

plt.figure(figsize=(12, 6))
monthly_avg.plot(kind='bar', ax=plt.gca())
plt.title('Monthly Average Contracts Issued and Expired')
plt.xlabel('Month')
plt.ylabel('Average Number of Contracts')
plt.xticks(range(12), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], rotation=45)
plt.legend(title='Metric')
plt.grid(axis='y')
plt.show()

# 8. 시각화 7: 발생 및 종료 건수의 이동 평균
df_recent['issued_moving_avg'] = df_recent['contract_issued'].rolling(window=6).mean()
df_recent['expired_moving_avg'] = df_recent['contract_expired'].rolling(window=6).mean()

plt.figure(figsize=(12, 6))
plt.plot(df_recent.index, df_recent['contract_issued'], label='Contracts Issued', alpha=0.5)
plt.plot(df_recent.index, df_recent['contract_expired'], label='Contracts Expired', alpha=0.5, linestyle='--')
plt.plot(df_recent.index, df_recent['issued_moving_avg'], label='Issued (6-Month Moving Avg)', linewidth=2, color='blue')
plt.plot(df_recent.index, df_recent['expired_moving_avg'], label='Expired (6-Month Moving Avg)', linewidth=2, color='orange')
plt.title('Contracts Issued and Expired with Moving Averages')
plt.xlabel('Month')
plt.ylabel('Number of Contracts')
plt.legend()
plt.grid()
plt.show()

In [2]:
# 날짜 형식 정제
def safe_to_datetime(date_str):
    try:
        return pd.to_datetime(date_str, errors='coerce')  # 비정상 날짜는 NaT로 처리
    except Exception as e:
        return pd.NaT

df['month'] = df['month'].apply(safe_to_datetime)

# 비정상적인 날짜를 포함한 행 제거
df = df.dropna(subset=['month'])

# 인덱스 설정
df.set_index('month', inplace=True)

# 최근 5년치 데이터만 선택
recent_years = 5
start_date = df.index.max() - pd.DateOffset(years=recent_years)
df_recent = df[df.index >= start_date]

# 데이터 시각화
plt.figure(figsize=(12, 6))
plt.plot(df_recent.index, df_recent['contract_issued'], label='Contracts Issued', marker='o')
plt.plot(df_recent.index, df_recent['contract_expired'], label='Contracts Expired', marker='o', linestyle='--')
plt.title(f'Monthly Contracts Issued and Expired (Last {recent_years} Years)')
plt.xlabel('Month')
plt.ylabel('Number of Contracts')
plt.legend()
plt.grid()
plt.show()

# ARIMA 모델링 및 예측
# 발생 건수에 대한 데이터 선택
issued_data = df_recent['contract_issued']

# ARIMA 모델 학습
arima_model_issued = ARIMA(issued_data, order=(1, 1, 1))  # (p, d, q) 값은 필요에 따라 조정
arima_result_issued = arima_model_issued.fit()

# 미래 데이터 예측 (12개월)
forecast_issued = arima_result_issued.forecast(steps=12)
forecast_index = pd.date_range(start=df_recent.index[-1] + pd.offsets.MonthBegin(1), periods=12, freq='MS')
forecast_issued = pd.Series(forecast_issued, index=forecast_index)

# 종료 건수에 대한 데이터 선택
expired_data = df_recent['contract_expired']
arima_model_expired = ARIMA(expired_data, order=(1, 1, 1))  # (p, d, q) 값은 필요에 따라 조정
arima_result_expired = arima_model_expired.fit()

# 미래 데이터 예측 (12개월)
forecast_expired = arima_result_expired.forecast(steps=12)
forecast_expired = pd.Series(forecast_expired, index=forecast_index)

# 7. 예측 결과 시각화
plt.figure(figsize=(12, 6))
plt.plot(df_recent.index, df_recent['contract_issued'], label='Contracts Issued (Actual)', marker='o')
plt.plot(df_recent.index, df_recent['contract_expired'], label='Contracts Expired (Actual)', marker='o', linestyle='--')
plt.plot(forecast_issued.index, forecast_issued, label='Contracts Issued (Forecast)', linestyle='-', color='blue')
plt.plot(forecast_expired.index, forecast_expired, label='Contracts Expired (Forecast)', linestyle='--', color='red')
plt.title(f'Forecasting Contracts Issued and Expired (Last {recent_years} Years)')
plt.xlabel('Month')
plt.ylabel('Number of Contracts')
plt.legend()
plt.grid()
plt.show()